<a href="https://colab.research.google.com/github/JoseRAdan/information_retrieval_eq2/blob/main/Avance3_Equipo2_ver03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AVANCE 3 EQUIPO 2
## PRYECTO INTEGRADOR


## ARQUITECTURA

Basado en la arquitectura genérica RETRIEVER-READER.

Usando la herramienta **Haystack** para validación del modelo de lenguage de **Hugging Face Hub** llamado "**dccuchile/albert-base-spanish-finetuned-qa-mlqa**" en su aplicabilidad de extraer respuestas acertadas del tipo de información que representa la normatividad académica.

## NOTAS PARA EJECUCION EN COLAB

Aunque corre sin GPU, es recomendable activarlo si esta disponible.

## Instalar Haystack

In [1]:
%%bash

pip install --upgrade pip
pip install farm-haystack[colab,inference]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.6/150.6 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 22.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 47.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 50.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.7/48.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 62.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## Puede ser necesario reiniciar después de instalar el siguiente

In [2]:
!pip install farm-haystack[elasticsearch]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 386.0/386.0 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.0.7
    Uninstalling urllib3-2.0.7:
      Successfully uninstalled urllib3-2.0.7


Set the logging level to INFO:

In [3]:
import logging

logging.basicConfig(format="%(levelname)s - %(name)s -  %(message)s", level=logging.WARNING)
logging.getLogger("haystack").setLevel(logging.INFO)

## Preparar fuente de datos

####Crear data/pdf
####Crear data/txt
####Crear data/txt_norm para guardar archivos con texto normalizado (pruebas)
####Crear data/txt_raw para guardar archivos con texto sin pre procesamiento
####Copiar PDFs en data/pdf
####Usar libreria PyPDF2 para extraer texto de archivos

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.3 MB/s eta 0:00:00


Limpieza de datos usando NTLK

In [5]:
#!pip install nltk
#import nltk
#import string
#nltk.download('stopwords')

In [6]:
#def clean_text(texto_extraido):
#  texto_limpio = nltk.corpus.stopwords.words("spanish") + ["¿", "¡", ".", ",", ":", ";", "!", "?", "\"", "\'", "\n", "\t"]
#  texto_sin_stopwords = "".join([palabra + ' ' for palabra in texto_extraido.split() if palabra not in texto_limpio])
#  texto_normalizado = texto_sin_stopwords.lower()#texto_sin_puntuacion.lower()
#  return texto_normalizado

## Extrayendo texto de PDF y guardarlos como archivos txt


In [7]:
import os
import PyPDF2

# Carpeta donde se encuentran los PDFs
carpeta_pdf = "/content/data/pdf"
carpeta_txt = "/content/data/txt"

# Recorre cada archivo PDF en la carpeta
for archivo in os.listdir(carpeta_pdf):
#    if archivo.endswith("Correo_Oficial_ITESM_0056_ED_047_Anexo1.pdf"):
    if archivo.endswith(".pdf"):
        # Extrae el nombre del archivo sin la extensión
        nombre_archivo, _ = os.path.splitext(archivo)

        # Abre el archivo PDF en modo lectura binaria
        with open(os.path.join(carpeta_pdf, archivo), 'rb') as pdf_file:
            # Crea un lector de PDF
            lector = PyPDF2.PdfReader(pdf_file)

            # Extrae el texto de cada página
            texto = ""
            texto_extraido = ""
            texto_raw = ""
            for pagina in lector.pages:
                # Extrae el texto de la página
                texto_pagina = pagina.extract_text()
                texto_raw += texto_pagina
                #texto_pagina_no_guiones = texto_pagina.replace('-\n', '').replace(' - \n', '').replace(' -\n', '').replace('- \n', '').replace('-  \n', '')
                texto_pagina_no_guiones = texto_pagina.replace(' -\n', '')

                # Juntar palabras incompletas por guiones
                palabras_sin_guiones = []
                palabra_actual = ""
                for caracter in texto_pagina_no_guiones:
                  if caracter == "-":
                    if palabra_actual:
                      palabras_sin_guiones.append(palabra_actual)
                    palabra_actual = ""
                  else:
                    palabra_actual += caracter

                if palabra_actual:
                  palabras_sin_guiones.append(palabra_actual)

                texto_pagina_sin_guiones = " ".join(palabras_sin_guiones)
                texto_extraido += texto_pagina_sin_guiones

#            texto_normalizado = clean_text(texto_extraido)

            # Guarda el texto en un archivo de texto con el nombre original
            with open(carpeta_txt + '/' + f"{nombre_archivo}.txt", "w", encoding="utf-8") as archivo_txt:
                archivo_txt.write(texto_extraido)
            # Guarda el texto raw en un archivo de texto con el nombre original
            with open(carpeta_txt + '_raw/' + f"{nombre_archivo}.txt", "w", encoding="utf-8") as archivo_txt:
                archivo_txt.write(texto_raw)

## Inicializar DocumentStore

We'll start creating our question answering system by initializing a DocumentStore. A DocumentStore stores the Documents that the question answering system uses to find answers to your questions. In this tutorial, we're using the `InMemoryDocumentStore`, which is the simplest DocumentStore to get started with. It requires no external dependencies and it's a good option for smaller projects and debugging. But it doesn't scale up so well to larger Document collections, so it's not a good choice for production systems. To learn more about the DocumentStore and the different types of external databases that we support, see [DocumentStore](https://docs.haystack.deepset.ai/docs/document_store).

Let's initialize the the DocumentStore:

In [8]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore(use_bm25=True)

INFO:haystack.telemetry:Haystack sends anonymous usage data to understand the actual usage and steer dev efforts towards features that are most meaningful to users. You can opt-out at anytime by manually setting the environment variable HAYSTACK_TELEMETRY_ENABLED as described for different operating systems in the [documentation page](https://docs.haystack.deepset.ai/docs/telemetry#how-can-i-opt-out). More information at [Telemetry](https://docs.haystack.deepset.ai/docs/telemetry).
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


##Indexar informacion de archivos txt

In [9]:
from haystack import Pipeline
from haystack.nodes import TextConverter, PreProcessor

indexing_pipeline = Pipeline()
text_converter = TextConverter()
preprocessor = PreProcessor(
    clean_whitespace=False,
    clean_header_footer=False,
    clean_empty_lines=True,
    split_by="word",
    split_length=200,
    split_overlap=20,
    split_respect_sentence_boundary=False,
)
#    language='es',

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [10]:
import os

indexing_pipeline.add_node(component=text_converter, name="TextConverter", inputs=["File"])
indexing_pipeline.add_node(component=preprocessor, name="PreProcessor", inputs=["TextConverter"])
indexing_pipeline.add_node(component=document_store, name="DocumentStore", inputs=["PreProcessor"])

files_to_index = [carpeta_txt + "/" + f for f in os.listdir(carpeta_txt)]
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing: 100%|██████████| 187/187 [00:02<00:00, 82.90docs/s]
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'eadec48c618c0f298844f98dbb20022e' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'ba52c8d9e20f160a503e85ac5e85f1ea' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '1c41183da56af1c5f4ced332b5ba9d4' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'bd7b7de977d85f27e56b4deecdb767c8' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '21b5a3f6b5dc937c5ce869d83ba2dc1c' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '8a03e5c9ceee56691022

{'documents': [<Document: {'content': 'Disposiciones Académicas para\nProgramas de Internacionalización de\nAlumnos del Tec de Monterrey\nVersión 1.0\nVicerrectoría de Internacionalización\nNoviembre, 2002Disposiciones Académicas para Programas  de Internacionalización\nSistema ITESM\n07 de noviembre de 2002\n1ÍNDICE\ni. INTRODUCCIÓN ........................................................................................................... 2\nii. ALCANCES DE ESTE DOCUMENTO ........................................................................ 2\n1. CONVENIOS Y ACUERDOS DE COOPERACIÓN ACADÉMICA\nINTERNACIONAL ............................................................................................................ 3\n1.1 De los criterios de afiliación del Tec de Monterrey .............................................. 3\n1.2 De la administración de convenios ....................................................................... 4\n1.3 De la generación y administración de programas 

In [11]:
import os
from haystack.pipelines.standard_pipelines import TextIndexingPipeline

files_to_index = [carpeta_txt + "/" + f for f in os.listdir(carpeta_txt)]
indexing_pipeline = TextIndexingPipeline(document_store)
indexing_pipeline.run_batch(file_paths=files_to_index)

INFO:haystack.pipelines.base:It seems that an indexing Pipeline is run, so using the nodes' run method instead of run_batch.
Preprocessing: 100%|██████████| 187/187 [00:03<00:00, 55.83docs/s]
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'aa0f2944548d34282bb145814cb03226' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '7cf2c07cb28a66dbe0c622cb48724f7' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'b0e220f7b996835437b3351b3c7fb8c0' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'a8e2511f843e1772e35b7458e313b8d2' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id 'e60b15376fc2a73b8ad5af3b909de4fe' already exists in index 'document'
INFO:haystack.document_stores.base:Duplicate Documents: Document with id '4be4202bb75f1ef0647c

{'documents': [<Document: {'content': 'Disposiciones Académicas para\nProgramas de Internacionalización de\nAlumnos del Tec de Monterrey\nVersión 1.0\nVicerrectoría de Internacionalización\nNoviembre, 2002Disposiciones Académicas para Programas  de Internacionalización\nSistema ITESM\n07 de noviembre de 2002\n1ÍNDICE\ni. INTRODUCCIÓN ........................................................................................................... 2\nii. ALCANCES DE ESTE DOCUMENTO ........................................................................ 2\n1. CONVENIOS Y ACUERDOS DE COOPERACIÓN ACADÉMICA\nINTERNACIONAL ............................................................................................................ 3\n1.1 De los criterios de afiliación del Tec de Monterrey .............................................. 3\n1.2 De la administración de convenios ....................................................................... 4\n1.3 De la generación y administración de programas 

## Initializar el Retriever

Our search system will use a Retriever, so we need to initialize it. A Retriever sifts through all the Documents and returns only the ones relevant to the question. This tutorial uses the BM25 algorithm. For more Retriever options, see [Retriever](https://docs.haystack.deepset.ai/docs/retriever).

Let's initialize a BM25Retriever and make it use the InMemoryDocumentStore we initialized earlier in this tutorial:

In [12]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store=document_store)

The Retriever is ready but we still need to initialize the Reader.

## Inicializar el Reader

El READER buscará textos recibidos del RETRIEVER y extraerá los candidatos más coinicidentes para lo cual usará FARMReader con el modelo de pregunta-respuesta de Hugging Face llamado "dccuchile/albert-base-spanish-finetuned-qa-mlqa".

In [13]:
from haystack.nodes import FARMReader

#reader = FARMReader(model_name_or_path="dccuchile/bert-base-spanish-wwm-uncased-finetuned-qa-mlqa", use_gpu=True)
reader = FARMReader(model_name_or_path="dccuchile/albert-base-spanish-finetuned-qa-mlqa", use_gpu=True)

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/842 [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model: * LOADING MODEL: 'dccuchile/albert-base-spanish-finetuned-qa-mlqa' (Albert)


pytorch_model.bin:   0%|          | 0.00/44.9M [00:00<?, ?B/s]

INFO:haystack.modeling.model.language_model:Auto-detected model language: spanish
INFO:haystack.modeling.model.language_model:Loaded 'dccuchile/albert-base-spanish-finetuned-qa-mlqa' (Albert model) from model hub.


tokenizer_config.json:   0%|          | 0.00/476 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.74M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

INFO:haystack.modeling.utils:Using devices: CPU - Number of GPUs: 0


We've initalized all the components for our pipeline. We're now ready to create the pipeline.

## Creacion del Pipeline Retriever-Reader


In [14]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

The pipeline's ready, you can now go ahead and ask a question!

## Probar haciendo una pregunta

1. Use the pipeline `run()` method to ask a question. The query argument is where you type your question. Additionally, you can set the number of documents you want the Reader and Retriever to return using the `top-k` parameter. To learn more about setting arguments, see [Arguments](https://docs.haystack.deepset.ai/docs/pipelines#arguments). To understand the importance of the `top-k` parameter, see [Choosing the Right top-k Values](https://docs.haystack.deepset.ai/docs/optimization#choosing-the-right-top-k-values).

In [18]:
prediction = pipe.run(
    query="¿Que es el período sabático?", params={"Retriever": {"top_k": 15}, "Reader": {"top_k": 5}}
)
#prediction = pipe.run(
#    query="¿Cuál es la definición exacta de un crédito académico en nuestra institución para los 3 niveles Preparatoria, Profesional y Posgrado?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)
#prediction = pipe.run(
#  query="¿Cuál es la calificación aprobatoria en nivel Posgrado?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)
#prediction = pipe.run(
#  query="¿Para qué niveles de estudio aplican los Lineamientos de Credenciales Alternativas?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)
#prediction = pipe.run(
#  query="¿Cuáles son los correos más recientes sobre créditos?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
#)
from pprint import pprint

pprint(prediction)

Inferencing Samples: 100%|██████████| 1/1 [00:08<00:00,  8.25s/ Batches]

{'answers': [<Answer {'answer': 'Profesional y Posgrado', 'type': 'extractive', 'score': 0.5959579944610596, 'context': 'o del Tecnológico de Monterrey \nque aplican para los niveles de Profesional y Posgrado, así como para Educación Continua, además de los \nlineamientos ', 'offsets_in_document': [{'start': 244, 'end': 266}], 'offsets_in_context': [{'start': 64, 'end': 86}], 'document_ids': ['f429395ef090e55be9425b1098bae991'], 'meta': {'_split_id': 0, '_split_overlap': [{'doc_id': 'e094bc0ed6775e31235497bcceb8a488', 'range': (939, 957)}]}}>,
             <Answer {'answer': 'Profesional, Posgrado y Educación Continua', 'type': 'extractive', 'score': 0.5419474244117737, 'context': 'nológico de\nMonterrey que aplican para los niveles de Profesional, Posgrado y Educación Continua, así como  los\nlineamientos para la apertura y cierre', 'offsets_in_document': [{'start': 198, 'end': 240}], 'offsets_in_context': [{'start': 54, 'end': 96}], 'document_ids': ['24f44066b254af031a60907f565f8d40'

In [19]:
from haystack.utils import print_answers

print_answers(prediction, details="all")  ## Choose from `minimum`, `medium`, and `all`

('Query: ¿Para qué niveles de estudio aplican los Lineamientos de Credenciales '
 'Alternativas?')
'Answers:'
[   <Answer {'answer': 'Profesional y Posgrado', 'type': 'extractive', 'score': 0.5959579944610596, 'context': 'o del Tecnológico de Monterrey \nque aplican para los niveles de Profesional y Posgrado, así como para Educación Continua, además de los \nlineamientos ', 'offsets_in_document': [{'start': 244, 'end': 266}], 'offsets_in_context': [{'start': 64, 'end': 86}], 'document_ids': ['f429395ef090e55be9425b1098bae991'], 'meta': {'_split_id': 0, '_split_overlap': [{'doc_id': 'e094bc0ed6775e31235497bcceb8a488', 'range': (939, 957)}]}}>,
    <Answer {'answer': 'Profesional, Posgrado y Educación Continua', 'type': 'extractive', 'score': 0.5419474244117737, 'context': 'nológico de\nMonterrey que aplican para los niveles de Profesional, Posgrado y Educación Continua, así como  los\nlineamientos para la apertura y cierre', 'offsets_in_document': [{'start': 198, 'end': 240}], 'offsets_